In [1]:
%load_ext autoreload
%autoreload 2

from utils.boolrank import DualEncoderModel
from utils.evaluation import evaluate_on_generated

model_name = 'BAAI/bge-small-en-v1.5'
# model_name = 'BAAI/llm-embedder'
# model_name = 'dmis-lab/biobert-v1.1'
model = DualEncoderModel(model_name)
model_name = model.model_name.split("/")[-1]
_ = model.eval()

In [2]:
from utils.data_processing import paths_to_dataset

bool_key = "bool_query"
nl_key = "nl_query"
data_path = "training"
path = "data/{}.jsonl"
pubmed = path.format(data_path)
TAR = path.format("TAR_data")
sysrev = path.format("sysrev_conv")
dataset = paths_to_dataset([pubmed, TAR, sysrev], test_only_sources=["TAR", "sysrev"], test_only=True)
# dataset = paths_to_dataset([TAR, sysrev], test_only_sources=["TAR", "sysrev"], test_only=True)
dataset

Removing similar: 100%|██████████| 5/5 [00:04<00:00,  1.19it/s]


DatasetDict({
    train: []
    test: {'pubmed-query': Dataset({
        features: ['nl_query', 'bool_query', 'source', 'quality', 'mission_hash', 'pmid', '__index_level_0__'],
        num_rows: 2100
    }), 'pubmed-searchrefiner': Dataset({
        features: ['nl_query', 'bool_query', 'source', 'quality', 'mission_hash', 'pmid', '__index_level_0__'],
        num_rows: 210
    }), 'raw-jsonl': Dataset({
        features: ['nl_query', 'bool_query', 'source', 'quality', 'mission_hash', 'pmid', '__index_level_0__'],
        num_rows: 90
    }), 'TAR': Dataset({
        features: ['nl_query', 'bool_query', 'source', '__index_level_0__'],
        num_rows: 60
    }), 'sysrev': Dataset({
        features: ['id', 'link_to_review', 'nl_query', 'search_name', 'Date_from', 'Date_to', 'bool_query', 'seed_studies', 'included_studies', 'edited_search', 'source', '__index_level_0__'],
        num_rows: 60
    })}
})

In [ ]:
from utils.evaluation import evaluate

def generate_plots(format="pdf"):
    for key, data in dataset["test"].items():
        res = evaluate(model, data[bool_key], data[nl_key], plot=True, title=key)
        res["plot1"].savefig(f"models/clip/{model_name}/{key}1.{format}", format=format)
        res["plot2"].savefig(f"models/clip/{model_name}/{key}2.{format}", format=format)

In [ ]:
data = dataset["test"]["TAR"]
logits = evaluate(model, data[bool_key], data[nl_key])['logits']

In [ ]:
logs = model(data[bool_key][15], data[nl_key][:30])['logits']
(-logs).argsort()

In [ ]:
print(data[1])
print((-logits[15]).argsort())
ahead = [24, 20, 25, 22, 15]
for d in ahead:
    print(data[d])

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Input data (from your prints)

# Ranking info
ahead = [24, 20, 25, 22, 15]  # 15 should be correct but ranked too low

cutoff = 100
# Build dataframe
rows = []
for idx in ahead:
    rows.append([data[idx]["nl_query"][:cutoff]])

df = pd.DataFrame(rows, columns=["NL Query"])

# Plot as table
fig, ax = plt.subplots(figsize=(10, 2))
ax.axis("off")

table = ax.table(
    cellText=df.values,
    colLabels=df.columns,
    cellLoc="left",
    loc='center',
    colColours=["lightgrey"]*3,
)

# Color rows by status
for i in range(1, len(df) + 1):
    if i == 5:
        color = "lightgreen"
    else:
        color = "lightcoral"
    table[(i, 0)].set_facecolor(color)

# plt.tight_layout()
plt.title('Bool Query: (' + data[15]["bool_query"][:cutoff] + '...)')
plt.savefig("nl_bool_query_ranking.pdf", format="pdf")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

cutoff = 100
# Build dataframe
rows = []
for idx in [1]:
    rows.append([data[idx]["nl_query"][:cutoff]])

df = pd.DataFrame(rows, columns=["NL Query"])

# Plot as table
fig, ax = plt.subplots(figsize=(10, 2))
ax.axis("off")

table = ax.table(
    cellText=df.values,
    colLabels=df.columns,
    cellLoc="left",
    loc='center',
    colColours=["lightgrey"]*3,
)

color = "lightgreen"
table[(1, 0)].set_facecolor(color)

# plt.tight_layout()
plt.title('Bool Query: (' + data[1]["bool_query"][:cutoff] + '...)')
plt.savefig("nl_bool_query_ranking2.pdf", format="pdf")
plt.show()

In [ ]:
# path = r"models\clip\biobert-v1.1\b4_lr8E-06_(pubmed-que_pubmed-sea_raw-jsonl)^2no[]\model.safetensors"
# model.load(path)
# generate_plots()

## Untrained

In [8]:
evaluate_on_generated(model)

Evaluating groups: 100%|██████████| 400/400 [00:11<00:00, 35.25it/s]


,model,spearman,pearson,f3_variance,best_rank
0,HuggingfaceH4,-0.0874,-0.1612,0.0717,0.3339
1,gpt-3.5-turbo-0125,-0.2233,-0.2590,0.0553,0.2303
2,gpt-3.5-turbo-1106,-0.1543,-0.1999,0.0566,0.2415
3,meta-llama,-0.1815,-0.1351,0.0741,0.3732
4,open-mixtral-8x7b,-0.2158,-0.2844,0.0765,0.3241
5,gpt-4o-mini,-0.3500,-0.3831,0.0678,0.2152
6,o1-2024-12-17,0.0353,0.1302,0.0502,0.4353
7,mistralai,-0.2102,-0.2190,0.0754,0.2301
8,open-mistral-7b,-0.2450,-0.2918,0.0783,0.2888
9,gpt-4-1106-preview,-0.4831,-0.3754,0.0296,0.2100


\begin{tabular}{lcccc}
\toprule
model & spearman & pearson & f3_variance & best_rank \\
\midrule
HuggingfaceH4 & -0.087400 & -0.161200 & 0.071700 & 0.333900 \\
gpt-3.5-turbo-0125 & -0.223300 & -0.259000 & 0.055300 & 0.230300 \\
gpt-3.5-turbo-1106 & -0.154300 & -0.199900 & 0.056600 & 0.241500 \\
meta-llama & -0.181500 & -0.135100 & 0.074100 & 0.373200 \\
open-mixtral-8x7b & -0.215800 & -0.284400 & 0.076500 & 0.324100 \\
gpt-4o-mini & -0.350000 & -0.383100 & 0.067800 & 0.215200 \\
o1-2024-12-17 & 0.035300 & 0.130200 & 0.050200 & 0.435300 \\
mistralai & -0.210200 & -0.219000 & 0.075400 & 0.230100 \\
open-mistral-7b & -0.245000 & -0.291800 & 0.078300 & 0.288800 \\
gpt-4-1106-preview & -0.483100 & -0.375400 & 0.029600 & 0.210000 \\
Average & -0.211500 & -0.217900 & 0.063600 & 0.288200 \\
\bottomrule
\end{tabular}



In [ ]:
evaluate_on_generated(model, ["id"])

## Model 1

In [ ]:
model.load(r"models\clip\bge-small-en-v1.5\b16_lr1E-05_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-11288\model.safetensors")
# model.load(r"models\clip\biobert-v1.1\b16_lr1E-05_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-14110\model.safetensors")
evaluate_on_generated(model)

In [ ]:
evaluate_on_generated(model, ["id"])

## Model 2

In [ ]:
model.load(r"models\clip\bge-small-en-v1.5\b16_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-11288\model.safetensors")
# evaluate_on_generated(model)

In [ ]:
evaluate_on_generated(model, ["id"])